# Python CCD Data Reduction

---

### Names: [Enter your names here]

**Before you do anything else, go to *File -> Save As* and change the filename to include your name or initials. Make any requested edits to that copy.**

This notebook walks through the basics of CCD data reduction within python. More detailed instructions (involving more powerful pieces of software) can be found in the CCD Data Reduction Guide written by Matt Craig and Lauren Chambers [https://mwcraig.github.io/ccd-as-book/00-00-Preface](https://mwcraig.github.io/ccd-as-book/00-00-Preface), which itself is based on the [IRAF CCD reduction](http://ircamera.as.arizona.edu/Astr_518/irafguid.pdf) and [stellar photometry](https://www.mn.uio.no/astro/english/services/it/help/visualization/iraf/daophot2.pdf) guides by Phil Massey. 

You will need various fits images for this lab, which can be found on the Astro Server. Look under Shared -> CCD_Data -> Astro211 and upload all of the files in this folder. 

**New Code**

* Open and close a fits file.
* Print info about a fits file.
* Access the data and the header within a fits file.
* Access the elements of a header.
* Set the x-axis range of a plot.
* Calculate the median along one axis of a multi-dimensional array.
* Read in just the header from a fits file.
* Add a new keyword to a fits header.
* Turn a multi-dimensional array into a one-dimensional array.
* `enumerate`
* Create a new HDU object
* Save the HDU object to a fits file
* `/=`, `*=`, `+=`, `-=`
* `array.shape`
* `np.zeros((dim1,dim2))`

### Pseudo-code

Trying to remember all of the exact syntax of various python statements can often get in the way of figuring out how to set up a program. Knowing whether or not to use a for loop is a separate problem than knowing the exact syntax of a for loop. Pseudo-code uses a mixture of code and plain english to represent a program in a way that avoids any concerns about syntax, but still makes clear how the program is designed to work.

As an example, lets consider the function from the first lab that took in an array of numbers and returned the sum of these numbers. The exact python code was:

 ```
 def sum_func(array_like):
    out_sum = 0
    for i in array_like:
        out_sum += i
    return out_sum
```

Converting to pseudo-code this becomes:

```
sum_func(an array of numbers)
  Set initial sum to 0
  For each element in input array of numbers:
    Add element to sum
  Return sum
```

From this pseudo-code you can see that you will need a variable that contains the sum, and you need to know the syntax for looping over the elements in a list. The pseudo-code doesn't worry about exact variable names, or exact syntax, but does include the for loop and the return statement. 

Now lets consider an example of going in the other direction (converting from psuedo code to actual code). In the first lab you wrote a function that would go through a list of students and their grades and sort the students as either proficient or struggling. The pseudo-code version of this algorithm looks like:

```
For student and grade in class list
  If grade is greater than average grade
    Add student to profcient student lest
  Else
    Add student to struggling student list
```

This pseudo-code makes it clear that we need to loop over every student in the list, and compare the grade of that student to the average for the class. Based on this comparison the student is either added to the proficient student list or the struggling student list.

From this psuedo-code you can imagine how to construct the actual piece of code. You will need lists for proficient students and struggling students, and you need to be able to add new entries to each of these lists. You will need to know the average grade for the class, as well as have a class list that contains names and grades (these may be defined earlier in the code). You will also need to figure out how to loop over the class list in a way that gives you access to both the grade and the student name. The exact python code for completing this task was:

```
proficient_studnts = []
struggling_students = []
for i in class_grades:
  if float(i[1]) > mean_score:
    proficient_students.append(i)
  else:
    struggling_students.append(i)
```

There are no formal rules for pseudo-code, or for how detailed to be when writing pseudo-code. Being too general isn't helpful (i.e., replacing the second example with `sort the students based on their grade` is not very helpful) but if you include too much detail then you end up just writing the function. Psuedo-code is best viewed as a tool to help you think about how a piece of code is structured, without worrying about the exact syntax. Include as much, or as little, detail as is helpful for you to understand how the code operates. 

### Reducing CCD Data

Any data taken at an observatory must go through a number of steps before it can be used for scientific analysis. This includes correcting for the bias level, the dark current, and inhomogeneities in the sensitivity across the detector. These contributions can be accounted for with the use of bias frames, dark frames, and flat field images. *Bias frames* are images with an exposure time of zero, and represent the number of counts that each pixel starts with before any contribution from the sky, or other sources of electrons. *Dark frames* represent the contribution from the thermal electrons; they are matched to the exposure time of the science image since the thermal signal increases with time. *Flat field frames* are images of a uniformly illuminated object. Any differences in the recorded signal between different pixels in a flat field image are most likely due to differences in the response rate of different pixels.

Bias and dark frames are subtracted from the science image, while a normalized flat field frame is divided off to produce a processed image. The formula for constructing a useable science image from a raw image is:

$science = \frac{raw - bias - dark}{flat}$

In [ ]:
# Load in the necessary packages
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

### Bias Frames (and basic fits handling in python)

First, lets read in a bias frame. We will look at the `CalR-0001Bias.fit` file.


We will use the `astropy` package, which includes the ability to read in fits files. Fits files are read into an HDUList object, which is a flexible data type that contains the data as well as the header. [More info about fits files in python can be found on the [astropy.io.fits documentation page.]((https://docs.astropy.org/en/stable/io/fits/)]

Below, after reading in the fits file into the variable `bias1`, we use the `.info()` function to print information about the fits file.

In [ ]:
from astropy.io import fits

# Read in a bias frame
bias1 = fits.open('CalR-0001Bias.fit')
bias1.info() # print basic info about the fits file

The output of the `.info()` function says that this fits file has one extension (since there is only one line of output), and has dimensions 1024x1024. The data and header for this extension can be accessed using the `.data` and `.header`, as shown below. If other extensions existed, they could be accessed in a similar manner, by simply changing the index. All of the files that you will look at today will only have one extension.

In [ ]:
bias_data = bias1[0].data
bias_header = bias1[0].header

The header is a dictionary and individual elements, such as the image type and the exposure time, can be accessed as with dictionaries. We can print off all of the keys to see what they are.

In [ ]:
print('Image Type: {}'.format(bias_header['imagetyp'])) #Image type header keyword
print('Exposure Time: {}'.format(bias_header['exptime'])) #Exposure time image keyword
print('---')
for key in bias_header.keys():
  print(key) #Print out the keys

The data themselves are a numpy array, and the flux in individual pixels can be accessed in the same way as with any two-dimensional array. As we saw based on the result from the `.info()` function the data has dimensions 1024x1024. We can verify the dimensions by looking at the shape of the array.

In [ ]:
print('Shape of the array: {}'.format(bias_data.shape))

Since it is a regular numpy array, we can access the flux values at various locations

In [ ]:
print('Flux in corner element: {}'.format(bias_data[0,0]))

The `convenience_functions.py` file, available on Glow, contains a function called `show_image` that can help in displaying images. 

In [ ]:
from convenience_functions import show_image #<- This loads the show_image function
show_image(bias_data,cmap='gray') 

As a bias frame, this image represents the state of the detector before it has been exposed to light. Variations in pixel value are partly due to read noise, as well as defects in the detector. We can look at the distribution of pixel values using the `hist` function from matplotlib, which we met in the first lab.

In [ ]:
plt.hist(bias_data.flatten(),10000,histtype='step')
plt.xlim(1300,1700) #<-- What does this line do?

> **Q:** To the above code block, add code to label the X-axis as 'Counts' and add the title 'Distribution of Bias Counts'.

> **Q:** What does the `plt.xlim(1300,1700)` line do? You can either look up the answer on the interwebs, use the help function to look at its docstring, or you can play around with the values to see what changes.
>
> **A: [your answer here]**

### Dark Frames

Now lets look at a dark frame.

> **Q:** As we did with the bias frame, read in the first dark frame (`CalR-0001Dark.fit`) and display the image using the `show_image` function.

In [ ]:
# Read in the dark frame, and display the image

The dark frame represents the contribution from thermal electrons. The image was taken with the shutter closed and an exposure time of sixty seconds. The pixel values are only slightly higher than the bias frame, suggesting that the dark current is not very high. We can calculate the dark current by subtracting a bias image, and then dividing the result by the exposure time.

> **Q:** The following function calculates the average dark current, in electrons per second, absed on a given dark frame. *But there are errors and missing lines in the function.* Correct the errors and missing lines so that the function runs properly.

In [ ]:
def calc_dark_current(dark_filename):
  dark_image = #Insert code that reads the fits file dark_filename into the variable dark_image
  dark_data = dark_image[0].data
  dark_header = dark_image[0].bias_header
  exp_time = #Insert code that reads the exptime keyword from the header, places this value in the variable exp_time
  dark_image.close()
  gain = 2.
  avg_dark_current = gain*np.mea(dark_data)/exp_time
  print('Average Dark current: {:0.2f} e-/sec'.format(avg_current)

calc_dark_current(CalR-0001Dark.fit)

> **Q:** Convert the function defined above from code into pseudo-code

```
[insert answer here, between the sets of three tick-marks]
```

### Flat Field

Now lets look at the flat field

> **Q:** As we did with the bias frame, read in the first flat field frame (`CalR-0001R.fit`). Display the image using the `show_image` function.

In [ ]:
# Read in a flat field, and display the image.

### Reducing science data

A science image needs to be corrected for the dark, bias, and flat field. This can be done based on individual dark, bias, and flat field images but it is much better to combine multiple frames. This is because each frame contains read noise on the individual pixels. By average multiple frames we can reduce the contribution of the read noise.

The following introduces how to combine multiple frames, and how to save the output. First we will set up a dictionary that contains the names of all of the calibration files. This includes the bias frames, the dark frames, and the flat field images for each filter.

In [ ]:
# Create a dictionary with the file names of the bias frames, dark frames, and flat field frames
data_files = {'biases':['CalR-00%02dBias.fit' % x for x in range(1,11)],
              'darks':['CalR-00%02dDark.fit' % x for x in range(1,11)],
              'Rflats':['CalR-00%02dR.fit' % x for x in range(1,6)]}

Next we will open each bias frame and save the data into one large master array (`bias_data`)

In [ ]:
from astropy.utils import data
# Read in the bias frames
bias_data = np.zeros((len(data_files['biases']),1024,1024)) # First create an empty numpy array with the right dimensions
for i,file in enumerate(data_files['biases']):
  image = fits.open(file)
  bias_data[i,:,:] = image[0].data
  image.close()

In the for loop above, I introduced the `enumerate` function. This function takes in a list and returns two items: the first is an index, and the second is the value of the list at that index. We can see how this works if I run a simple for loop that prints out the two variables.

In [ ]:
for i,file in enumerate(data_files['biases']):
  print(i,file)

As it steps through the elements of the list, the `enumerate` function returns the index at each position, and the value of the list at that position. When employed in the for loop, this allows us to use the file name for reading in the fits file, and then use the index when placing the data within our combined array.

Now lets return to the `bias_data` variable. This is a three dimensional array, with dimensions (Number of bias frames) x (Number of rows) x (Number of columns). As with the numpy array, individual elements can be specified using square brackets, but now three indeces need to be specified. The for loop above puts each image into a different portion of the array, so that all of the data is accessible in one array.

> **Q:** Which of the following lines of code selects the center pixel of the 1st image? (Delete the incorrect answers)
>
> `bias_data[512,512,1]`
>
> `bias_data[1,512,512]`
>
> `bias_data[0,512,512]`
>
> `bias_data[512,512,0]`

Note that for a 2D image the x-axis (FITS NAXIS1) is the *second* index, while the y-axis is the first index. Selecting out the flux at e.g. x=10, y=45 in the first image would use the code `bias_data[0,44,9]`.

> **Q:** Which of the following selects out the pixels in the first image with 220 < x < 240 and 450 < y < 470? (Delete the incorrect answers)
>
> `bias_data[220:240,450:470,0]`
>
> `bias_data[450:470,220:240,0]`
> 
> `bias_data[0,220:240,450:470]`
>
> `bias_data[0,450:470,220:240]`


Next we need to combine all of these images into one image. At each pixel position we want to take the median across all of the images. In this way we construct a new image that has the same dimensions as the original image, but has less noise. This could be done use two for loops (one looping over the rows and the other looping over columns), but the median function within numpy has this functionality built in. The following line of code will take the median along the 1st axis at each position in the 2nd and 3rd axis (e.g., at each pixel position, take the median across all of the images).

In [ ]:
combined_bias = np.median(bias_data,axis=0) #<- The axis keyword starts counting at zero

Now lets look at a single bias frame and the combined bias frame side by side.

In [ ]:
show_image(bias_data[0,:,:],cmap='gray')
plt.title('One Bias Frame')

show_image(combined_bias,cmap='gray')
plt.title('Combined Bias Frame')

Notice how the combined bias frame looks less grainy. That is because the graininess was due to read noise, which is reduced when taking a median combination of multiple frames.

Next we will write out the median-combined bias frame so that we can access it later.

In [ ]:
# Write out the combined bias file

# First read in the header from one of the bias files, and add a comment to say that this is a combined file
hdr = fits.getheader('CalR-0001Bias.fit') #<- Read in just the header. This saves time and memory
hdr['history'] = 'Median combination bias frame' #<- Add a new keyword, with the given value (just like any other dictionary).

# Create a new HDU object
hdu = fits.PrimaryHDU(combined_bias,hdr)
hdu.writeto('combined_bias.fits',overwrite=True)

Now we want to do the same thing for the dark and flat field frames. The procedure is very similar, with two additional considerations:


*   We must subtract the bias level from each dark frame before they are median-combined. We want the final combined dark image to only represent the dark current, not the dark current plus the bias level.
*   Each flat image must have the bias level subtracted, as well as the dark current. After created a combined flat field image, we then need to normalize this combined image, which we can do by dividing the image by its median value.






> **Q:** In the space below, write out the psuedo-code for processing the flat field frames.

```
[insert answer here]
```



Lets start with the dark frame.

> **Q:** The code below gives the basic outline for creating a median-combined, bias-subtracted dark frame. Insert the necessary lines of code to complete this code block. (There are no intentional errors in this code block, just missing pieces of code.)


In [ ]:
# Now lets do the same for the dark frame, making sure to subtract of the bias

# Read in the dark frames
dark_data = np.zeros((len(data_files['darks']),1024,1024))
for i,file in enumerate(data_files['darks']):
  image = fits.open(file)
  dark_data[i,:,:] = #Insert code to input the dark image data, while subtracting off the bias
  image.close()

# Insert code to median combine the dark_data array
combined_dark = 

# Write out the combined dark file
# First read in the header from one of the dark files, and add a comment to say that this is a combined file
hdr = fits.getheader('CalR-0001Dark.fit')
hdr['history'] = 'Median combined dark frame'

# Create an HDU object from the data and header, and write out the result.
hdu = fits.PrimaryHDU(combined_dark,hdr)
hdu.writeto('combined_dark.fits',overwrite=True)

In [ ]:
show_image(dark_data[0,:,:],cmap='gray')
plt.title('One Dark Frame')

show_image(combined_dark,cmap='gray')
plt.title('Combined Dark')

> **Q:** Now repeat for the flat field images. Make sure to subtract off both the bias level and the dark current. The dark current must be scaled to the exposure time of the flat field images. To do this, divide the combined dark current image by the exposure time of the dark current image (120 seconds) to get the dark current per second, and then multiply by the exposure time of the flat field image.

In [ ]:
from astropy.io.fits import header
flat_data = np.zeros((len(data_files['Rflats']),1024,1024))
for i,file in enumerate(data_files['Rflats']):
  image = fits.open(file)
  header = image[0].header
  exptime = #Insert code to read in the exposure time from the header
  flat_data[i,:,:] = #Insert code to read the data from the image, and subtract both the bias and the scaled dark
  image.close()

# Insert code to median combine the flat data, and place the results into a variable called combined_flat
combined_flat = 
# Next, normalize the flat field image.
#    Both of the following statements are equivalent (we keep one commented out because we don't want to divide twice)
combined_flat = combined_flat/np.median(combined_flat)
#combined_flat /= np.median(combined_flat)

# First read in the header from one of the flat files, and add a comment to say that this is a combined file
hdr = fits.getheader('CalR-0001R.fit')
hdr['history'] = "Median combined flat frame"

#Insert code to create a new HDU object, using the combined_flat and hdr.

#Save the result as combined_Rflat.fits

In [ ]:
show_image(flat_data[0,:,:],cmap='gray')
plt.title('One Flat Frame')

show_image(combined_flat,cmap='gray')
plt.title('Combined Flat')

Now that we have our master bias, dark, and flat field images, we can apply them to a science image.

> **Q:** Below is the beginning of a function that will read in a science image, and then perform the bias/dark/flat corrections. We want to write a function that takes as inputs the filename of the raw image (as a string), the filename of the output file (a string, and it should be different from the original filename), and the name of the bias, dark, and flat field frames. The function reads in the bias, dark, flat field, and science images, performs the necessary calibrations to the science image, and then writes out the calibrated science frame.
>
> First write out the psuedo-code for this function in the space below.


```
def process_data():
  [insert answer here]
```


> **Q:** Next, convert from your pseudo-code to actual code in the space below.

In [ ]:
def process_data(filename,new_filename,bias_file='combined_bias.fits',dark_file='combined_dark.fits',flat_file='combined_Rflat.fits'):

  science_image = # Insert code here
  science_data = #Insert code here
  hdr = #Insert code here
  exptime = #Insert code here
  

  bias_image = #Insert code here
  bias_data = #Insert code here


  dark_image = #Insert code here
  dark_data = #Insert code here


  flat_image = #Insert code here
  flat_data = #Insert code here


  proc_data = #Insert code here

  #Insert code here
  hdu.writeto(new_filename,overwrite=True)
  


Test your code using the following code block (make sure to upload the necessary fits file from Glow to your working directory)

In [ ]:
#Test your code using this code block
process_data('M101_MartiC_180-0001.fit','M101_MartiC_180-0001_proc.fits')

old_image = fits.open('M101_MartiC_180-0001.fit')
old_data = old_image[0].data
old_image.close()

new_image = fits.open('M101_MartiC_180-0001_proc.fits')
new_data = new_image[0].data
new_image.close()

show_image(old_data,cmap='gray')
plt.title('Raw Image')

show_image(new_data,cmap='gray')
plt.title('Processed Image')

### Putting it all together

Now that you know how to process data, you can start to do some analysis with processed data. Let's look at the supernova in M101 from 2023. The files M101_1.fit, M101_2.fit, M101_3.fit, M101_4.fit, and M101_5.fit are images of M101, one taken before the supernova exploded, and the rest taken after the supernova exploded. Use these images to measure how the brightness of the supernova changed with time.

To do this:

1. Process each of the images using the function that you created.

2. Sum up the flux in a small box in a region around the supernova. The box size should be large enough to encompass the supernova, but not include too much contaminating flux from surrounding material in the galaxy. Use the same box size for all three images. For the pre-supernova image, use the location where the supernova shows up in the subsequent images.

3. From the header of each image, find the date when the image was collected.

4. Plot the flux of the supernova as a function of the number of days since the first image. Make sure to label the axes of the plot. 


*BONUS: Use loops to simplify your code.*

#### To turn in this lab, either email me (kmf4) a copy of this notebook, or place your copy in Shared/Astro211_F25/CCD_Reduction folder in the Astro server.